# Create Biomarker Supervisor Agent
In this notebook we create the Biomarker Supervisor Agent that will interact with all of the Biomarker sub-agents using the 'multi-agent-collaboration' feature in Bedrock Agents

#### Install Strands agents and required dependencies

In [ ]:
%pip install strands-agents strands-agents-tools boto3 requests --quiet

#### Ensure the latest version of boto3 is shown below
Ensure the boto3 version printed below is **1.39.3** or higher.

In [ ]:
%pip show boto3

## Prerequisites

1. Deploy the CloudFormation stack located at https://github.com/aws-samples/amazon-bedrock-agents-cancer-biomarker-discovery to your AWS account.
2. Run through the notebook environment setup in 00-setup-environment.ipynb.
3. Create all Biomarker sub-agents: 'Biomarker-database-analyst', 'Clinical-evidence-researcher', 'Medical-imaging-expert', and 'Statistician' using the notebooks 1 to 4.

#### Import required libraries

In [ ]:
import boto3
import json
import uuid
import requests
from typing import Dict, Any
from strands import Agent, tool
from strands.models import BedrockModel

from biomarker_agent import *
from clinical_research_agent import *
from medical_imaging_agent import *
from statistician_agent import *


# Agent Creation
In this section we create the supervisor agent

### Agents as Tools with Strands Agents

"Agents as Tools" is an architectural pattern in AI systems where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

1. A primary "orchestrator" agent handles user interaction and determines which specialized agent to call
2. Specialized "tool agents" perform domain-specific tasks when called by the orchestrator

This approach mimics human team dynamics, where a manager coordinates specialists, each bringing unique expertise to solve complex problems. Rather than a single agent trying to handle everything, tasks are delegated to the most appropriate specialized agent.

#### Lab 1: biomarker_database_analyst_agent tool

In [ ]:
@tool
def biomarker_database_analyst_agent(query: str) -> str:
    """
    Create biomarker query engine with redshift using Strands framework

    Args:
        query: An information request from the biomarker database

    Returns:
        A summary of the understanding of the user's query and the response.
    """
    try:
        # print(f"query: {query}")
        biomarker_agent = Agent(
            model=bedrock_model,
            tools=[get_schema, query_redshift, refine_sql],
            system_prompt=biomarker_agent_instruction
        )
        biomarker_agent_response = biomarker_agent(query)
        print("Biomarker agent Response:")
        print(biomarker_agent_response)
        return biomarker_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

#### Lab 2: clinical_evidence_research_agent tool

In [ ]:
@tool
def clinical_evidence_research_agent(query: str) -> str:
    """
    Research internal and external evidence using Strands framework

    Args:
        query: An information request from the clinical evidence

    Returns:
        Clinical evidence.
    """
    try:
        # print(f"query: {query}")
        clinical_research_agent = Agent(
            model=bedrock_model,
            tools=[query_pubmed, retrieve],
            system_prompt=clinical_research_agent_instruction
        )
        clinical_research_agent_response = clinical_research_agent(query)
        print("Clinical research agent Response:")
        print(clinical_research_agent_response)
        return clinical_research_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

#### Lab 3: clinical_evidence_research_agent tool

In [ ]:
@tool
def medical_imaging_agent(query: str) -> str:
    """
    Medical research assistant AI specialized in processing medical imaging scans of patients

    Args:
        query: Patient information to be used to compute imaging biomarkers

    Returns:
        The results of the medical imaging jobs.
    """
    try:
        # print(f"query: {query}")
        medical_imaging_agent = Agent(
            model=bedrock_model,
            tools=[compute_imaging_biomarker, analyze_imaging_biomarker],
            system_prompt=medical_imaging_agent_instruction
        )
        medical_imaging_agent_response = medical_imaging_agent(query)
        print("Medical imaging agent Response:")
        print(medical_imaging_agent_response)
        return medical_imaging_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

#### Lab 4: statistician_agent tool

In [ ]:
@tool
def statistician_agent(query: str) -> str:
    """
    Medical research assistant AI specialized in survival analysis with biomarkers

    Args:
        query: Information to be used to generate the chart

    Returns:
        A summary of your understanding of the user's query.
    """
    try:
        # print(f"query: {query}")
        statistician_agent = Agent(
            model=bedrock_model,
            tools=[create_bar_chart, plot_kaplan_meier, fit_survival_regression],
            system_prompt=statistician_agent_instruction
        )
        statistician_agent_response = statistician_agent(query)
        print("Statistician agent Response:")
        print(statistician_agent_response)
        return statistician_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

#### Define orchestrator agent configuration below

In [ ]:
agent_name = "multi-agent-biomarker"
agent_description = "Multi-agent collaboration for biomarker discovery"
agent_instruction = """You are a medical research assistant AI specialized in cancer biomarker analysis and discovery. 
Your primary task is to interpret user queries, use relevant agents for specific tasks, and provide consolidated medical insights based on the data. 
Use only the appropriate agents as required by the specific question. You can provide responses from a prior agent to the next agent 
in sequence. To analyze patient biomarkers data, you can retrieve relevant records from the database. 
To find the p-value of biomarkers, 
a. You need to query and store all records including survival status, survival duration in years, and the required biomarkers and 
b. You need to fit a surival regression model with that data in S3. 
When providing your response:
a. Start with a brief summary of your understanding of the user's query. 
b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required. 
c. Present the results of individual agents 
d. Conclude with a concise summary of the findings and their potential implications for medical research. 

Make sure to explain any medical or statistical concepts in a clear, accessible manner.

"""

#### Instantiate agent with the desired configuration

In [ ]:
# Strands Agents SDK allows easy integration of agent tools
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    region_name=region,
    temperature=0.1,
    streaming=False
)

try:
    orchestrator = Agent(
        model=bedrock_model,
        system_prompt=agent_instruction,
        callback_handler=None,
        # associate sub-agents to supervisor
        tools=[biomarker_database_analyst_agent, clinical_evidence_research_agent, medical_imaging_agent, statistician_agent]
    )
    
    print(f"Successfully created orchestrator agent: {agent_name}")
    
except Exception as e:
    print(f"Error creating agent: {e}")
    raise

## Ask supervisor agent different question types now that sub-agents are ready!

In [ ]:
# ---------------------------- Sample Question Bank --------------------------------------------

# Redshift Agent Questions
redshift_agent_query_1 = "How many patients are current smokers?"
redshift_agent_query_2 = "What is the average age of patients diagnosed with Adenocarcinoma?"

# Research Evidence Agent Questions
research_evidence_agent_query_1 = "Can you search PubMed for evidence around the effects of biomarker use in oncology on clinical trial failure risk?"
research_evidence_agent_query_2 = "What are the FDA approved biomarkers for non small cell lung cancer?"

# Medical Imaging Agent Questions (must run in sequence)
medical_imaging_agent_query_1 = "Can you compute the imaging biomarkers for the 2 patients with the lowest gdf15 expression values?"
medical_imaging_agent_query_2 = "Can you higlight the elongation and sphericity of the tumor with these patients. Can you depict images of them?"

# Scientific Analysis Agent Questions
scientific_analysis_agent_query_1 = "What is the best gene biomarker (lowest p value) with overall survival for patients that have undergone chemotherapy, Generate a bar chart of the top 5 gene biomarkers based on their p value and include their names in the x axis.?"

# Followup research evidence agent questions
research_evidence_agent_query_3 = "According to literature evidence, what metagene cluster does gdf15 belong to"
research_evidence_agent_query_4 = "What properties of the tumor are associated with metagene 19 activity and EGFR pathway"

# -----------------------------------------------------------------------------------------
test_query = redshift_agent_query_1 # Change value here to test different questions

print(f"Testing orchestrator agent with query: {test_query}")
print("=" * (39 + len(test_query)))

try:
    # Run the agent
    response = orchestrator(test_query)
    
except Exception as e:
    print(f"Error during agent execution: {e}")
    import traceback
    traceback.print_exc()

### Now the Biomarker Supervisor Agent is ready to assist you!